Подготовка пакета PySyft.

In [9]:
import os
from os import listdir, makedirs, remove
from os.path import exists, join, basename
import tarfile
import subprocess
from six.moves import urllib

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.utils as vutils
from torch.utils.data import DataLoader

import numpy as np

Подключение к владельцу данных.

In [ ]:
import syft as sy
duet = sy.join_duet("80132c82b91f1a19df2660ead4b3c11a")
sy.logging(file_path="./syft_ds.log")

In [2]:
import torch
rem_torch = duet.torch

Параметры Anaconda3 is already installed in /opt/anaconda3. Use 'conda update anaconda3' to update Anaconda3.

In [17]:
args = {
    "batch_size": 1000,
    "log_batch_size": 1000,
    "epochs": 10,
    "lr": 1.0,
    "gamma": 0.7,
    "no_cuda": False,
    "dry_run": False,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "save_model": True,
}

Список доступных тензоров

In [4]:
duet.store.pandas

,ID,Tags,Description
0,<UID: ba3b9930de0d4c1cb87410d34917b890>,[Train X],
1,<UID: b732b4b1d8e94cedb6b717a8dacb034c>,[Train Y],
2,<UID: f54fffd45c6b4d45a65c6cb03a3e40da>,[Test X],
3,<UID: 8a3715a47304442d9ded5f9cd295614e>,[Test Y],
4,<UID: 1c1fd71456224dbbbdae6ed87d28951f>,[train_num],


Загрузим удаленные данные

In [27]:
X_train = duet.store["Train X"]
y_train = duet.store["Train Y"]
train_num = duet.store["train_num"]
print(train_num)

AuthorizationException: You do not have permission to .get() Object with ID: <UID: 1c1fd71456224dbbbdae6ed87d28951f>Please submit a request.

In [19]:
class DatasetFromPointer(data.Dataset):
    def __init__(self, 
                 X_tensorpointer,
                 y_tensorpointer,
                 datanum_pointer,
                 ):
        super(DatasetFromPointer, self).__init__()
        self.X_tensorpointer = X_tensorpointer
        self.y_tensorpointer = y_tensorpointer
        self.datanum_pointer = datanum_pointer

    def __getitem__(self, index):
        input = self.X_tensorpointer[index]
        target = self.y_tensorpointer[index]
        return input, target

    def __len__(self):
        return self.datanum_pointer.get(
        request_block=True,
        reason="To write the training loop",
        timeout_secs=0,
        delete_obj=False,
    )
        
def batch_idx_fn(batch):
    return batch[0]

In [26]:
train_set = DatasetFromPointer(X_train, y_train, train_num)
len(train_set)

AttributeError: 'function' object has no attribute 'get'

In [20]:
train_set = DatasetFromPointer(X_train, y_train, train_num)
training_data_loader = DataLoader(dataset=train_set, num_workers=0, batch_size=args["batch_size"], shuffle=True, collate_fn=batch_idx_fn)

TypeError: 'NoneType' object cannot be interpreted as an integer

Зададим нашу сеть

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.lin1 = self.torch_ref.nn.Linear(X_train.shape[1],X_train.shape[1]*2)
        self.lin2 = self.torch_ref.nn.Linear(X_train.shape[1]*2, 1)
       
    def forward(self, x):
        x = self.torch_ref.nn.functional.relu(self.lin1(x))
        output = self.torch_ref.nn.functional.sigmoid(self.lin2(x))
        return output

In [ ]:
local_net = SyNet(rem_torch)
remote_net = model.send(duet)

Проверка CUDA

In [ ]:
has_cuda = False
has_cuda_ptr = rem_torch.cuda.is_available()
has_cuda = bool(has_cuda_ptr.get(
    request_block=True,
    name="cuda_is_available",
    reason="To run test and inference locally",
    timeout_secs=5,  # change to something slower
))
print(has_cuda)

use_cuda = not args["no_cuda"] and has_cuda
# now we can set the seed
rem_torch.manual_seed(args["seed"])

device = rem_torch.device("cuda" if use_cuda else "cpu")
print(f"Data Owner device is {device.type.get()}")

In [ ]:
# if we have CUDA lets send our model to the GPU
if has_cuda:
    model.cuda(device)
else:
    model.cpu()

Обучение модели

In [ ]:
params = model.parameters()
optimizer = rem_torch.optim.SGD(params, lr=args["lr"])
scheduler = rem_torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=args["gamma"])
criterion = rem_torch.nn.functional.binary_cross_entropy()

In [ ]:
for epoch in range(args["epochs"]):
    
    remote_net.train()
    epoch_loss = 0
    for batch_idx, data_pointers in enumerate(training_data_loader):

        optimizer.zero_grad()
        data_ptr, target_ptr = data_pointers[0], data_pointers[1]
        data_ptr_reshape = remote_torch.unsqueeze(remote_torch.unsqueeze(data_ptr, 0), 0)
        target_ptr_reshape = remote_torch.unsqueeze(remote_torch.unsqueeze(target_ptr, 0), 0)
       
        output_ptr = remote_net(data_ptr_reshape)
        
        loss = criterion(output_ptr, target_ptr_reshape)
        loss.backward()
        optimizer.step()

        if batch_idx % args["log_batch_size"] == 0:
            loss_item = loss.item().get(
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=3,
                delete_obj=False,
                verbose=False
                )
            print(f"epoch {epoch}, batch_idx {batch_idx}, loss {loss_item}")

        if args["dry_run"]:
            break

Сохранить модель

In [ ]:
remote_net.get(
    request_block=True,
    reason="test evaluation",
    timeout_secs=5
).save(f"model/small_signal_dataset.pt")